In [1]:
import pyrealsense2
from realsense_depth import *
import numpy as np
import open3d as o3d
from scipy import spatial



# Initialize Camera Intel Realsense
dc = DepthCamera()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [43]:
def visualize_rgbd(color_frame,depth_frame,from_depth=False):

    #rs-sensor-control   <<< Use this to get calibrations, get,set parameters etc...
    #rs-sensor-control is commandline tool by realsense 2.0 SDK, run from terminal
    
    #print(rgbd_image)
    #intrinsic.intrinsic_matrix =  [[602.71783447, 0.0, 313.06835938], [0.0, 601.61364746, 230.37461853], [0.0, 0.0, 1.0]]
    '''
     Intrinsic of "Color" / 640x480 / {YUYV/RGB8/BGR8/RGBA8/BGRA8/Y8/Y16}
      Width:      	640
      Height:     	480
      PPX:        	313.068359375
      PPY:        	230.374618530273
      Fx:         	602.717834472656
      Fy:         	601.613647460938
      Distortion: 	Inverse Brown Conrady
      Coeffs:     	0  	0  	0  	0  	0  
      FOV (deg):  	55.93 x 43.49
    '''
    
    #From camera  Intrinsic of "Depth" / 640x480 / {Z16} 
    #Distortion: Brown Conrady  Coeffs:  0  0  0  0  0   FOV (deg): 79.93 x 64.3
    w = 640
    h = 480
    fx = 381.838836669922
    fy = 381.838836669922
    cx = 320.060424804688
    cy = 237.036407470703
    
    #Depth frame ROS camera
    #w = 640
    #h = 480
    #fx = 347.99755859375
    #fy = 347.99755859375
    #cx = 320.0
    #cy = 240.0    
    
    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)
    #intrinsic.intrinsic_matrix = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]  #alternative method
    '''   
    extr_T = [0.0150703,0.000300758,9.12806e-06] #Depth to RGB
    extr_R = [[0.999992,0.00279048,0.00277953],
              [-0.00280054,0.99999,0.00362144],
              [-0.0027694,-0.00362919,0.99999]]  '''

    
    extr_T = [-0.0150693,-0.000342775,-5.21054e-05]   #RGB to Depth
    extr_R = [[0.999992,-0.00280054,-0.0027694],
              [0.00279048,0.99999,-0.00362919],
              [0.00277953,0.00362144,0.99999]]  
    
    cam.extrinsic = np.array([[extr_R[0][0], extr_R[0][1], extr_R[0][2], extr_T[0]], 
                                 [extr_R[1][0], extr_R[1][1], extr_R[1][2], extr_T[1]], 
                                 [extr_R[2][0], extr_R[2][1], extr_R[2][2], extr_T[2]], 
                                 [0.          , 0.          , 0.          , 1.      ]])

    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)))
    
    if from_depth==False:
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
        #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
    else:
        pcd =  o3d.geometry.PointCloud.create_from_depth_image(depth_raw, cam.intrinsic, cam.extrinsic, 
                                                               depth_scale=1000.0, depth_trunc=1000.0, 
                                                               stride=1, project_valid_depth_only=True)
    
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.
    return pcd



print("Loaded!!")

Loaded!!


In [44]:
#Check center pixel distance

spacing = 0.01 #spacing between each point in point cloud in meters (use this to approximate the shape of surface)
offset_y = 0.13 #distance to crop in world's X axis away from robot, to avoid robot's shadow appearing in PC.
offset_z = 0#0.05
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])

point = (320, 240)

_, depth_frame, color_frame = dc.get_frame()
print("color frame:",color_frame.shape, " Depth frame:",depth_frame.shape)
distance = depth_frame[point[1], point[0]]
print(distance)

pt_cloud = visualize_rgbd(color_frame,depth_frame,from_depth=False)

o3d.visualization.draw_geometries([pt_cloud,mesh])

downpcd = pt_cloud.voxel_down_sample(voxel_size=spacing)

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30)) #radius in meters
print("Align normals towards camera")
downpcd.orient_normals_towards_camera_location(camera_location=[0, 0, 0])

#o3d.visualization.draw_geometries([downpcd,mesh], point_show_normal=True)

## Filter out hidden points
diameter = np.linalg.norm(np.asarray(downpcd.get_max_bound()) - np.asarray(downpcd.get_min_bound()))
cam = [0, 0, diameter]
radius = diameter * 100
_, pt_map = downpcd.hidden_point_removal(cam, radius) #Get all points that are visible from given view point
downpcd = downpcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([downpcd,mesh], point_show_normal=True)


#v2_camera = np.array(downpcd.normals)
v2_camera_pts = np.array(downpcd.points)


#idx = np.where(abs(v2_camera_pts[:,2]) < (distance/1000)+0.1)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec
idx = np.where(abs(v2_camera_pts[:,2]) < abs(np.min(v2_camera_pts[:,2]))-0.06)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec

#print(idx)
filtered_Pt_Cloud = downpcd.select_by_index(idx, invert=False)


#CROP TO FILTER OUT ROBOT'S SHADOW ADJUST OFFSET ACCORDINGLY
PC_BBOX = filtered_Pt_Cloud.get_axis_aligned_bounding_box()
print(PC_BBOX)
minB_X = PC_BBOX.min_bound[0]
maxB_X = PC_BBOX.max_bound[0]
minB_Y = PC_BBOX.min_bound[1]
maxB_Y = PC_BBOX.max_bound[1]
minB_Z = PC_BBOX.min_bound[2]
maxB_Z = PC_BBOX.max_bound[2]
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, minB_Y, minB_Z+offset_z), max_bound=(maxB_X, maxB_Y-offset_y, maxB_Z)) 
filtered_Pt_Cloud = filtered_Pt_Cloud.crop(bbox)

Re = filtered_Pt_Cloud.get_rotation_matrix_from_xyz((0, np.pi, np.pi))
filtered_Pt_Cloud = filtered_Pt_Cloud.rotate(Re, center=(0,0,0))

print(filtered_Pt_Cloud)
o3d.visualization.draw_geometries([filtered_Pt_Cloud,mesh], point_show_normal=True)



nor = np.array(filtered_Pt_Cloud.normals)
pts = np.array(filtered_Pt_Cloud.points)

print()
distance,index = spatial.KDTree(pts).query( filtered_Pt_Cloud.get_center() ) #find coordinates 
                                                                             #that are closest to the center 

print(distance)
print(index)
print()
print("Center Coordinates(ground truth):",filtered_Pt_Cloud.get_center())
print("Points Coordinates(estimated center point):",pts[index])
print("Normal Coordinates(Normal of estimated center point):",nor[index])

color frame: (480, 640, 3)  Depth frame: (480, 640)
281
Recompute the normal of the downsampled point cloud
Align normals towards camera
AxisAlignedBoundingBox: min: (-0.430164, -0.251526, -0.778775), max: (0.497502, 0.47636, -0.186196)
PointCloud with 3649 points.

0.042753451712883736
1666

Center Coordinates(ground truth): [-0.07848412 -0.14323337  0.61660338]
Points Coordinates(estimated center point): [-0.08274019 -0.1462075   0.65904037]
Normal Coordinates(Normal of estimated center point): [ 0.17731347  0.01191491 -0.9840823 ]
